In [ ]:
from ift6758.data import load_events_dataframe
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from PIL import Image
from scipy.ndimage import gaussian_filter
from matplotlib.colors import BoundaryNorm
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import os

rink_width = 550
rink_height = 468
rink_x_min, rink_x_max = 0, 100
rink_y_min, rink_y_max = -42.5, 42.5
rink_image_path = "../figures/nhl_rink_cropped.png"

def transform_coordinates(x, y):
    transformed_x = ((x - rink_x_min) / (rink_x_max - rink_x_min)) * rink_width
    transformed_y = rink_height - ((y - rink_y_min) / (rink_y_max - rink_y_min) * rink_height)
    return transformed_x, transformed_y

def get_teams_dict(teams):
    teams_dict = {}
    for i in range(len(teams)):
        teams_dict[teams[i]] = i
    return teams_dict

def put_right_format(array):
    final_matrix = []
    for elem in array:
        new_elem = np.array([[elem] * (551)] * (468))
        final_matrix.append(new_elem)
    return np.array(final_matrix)

def get_adv_visuals(year: int, team: str):
    # Check if Golden Knights are selected for 2016
    if year == 2016 and team == "Golden Knights":
        return "Aucune donnée disponible pour cette équipe à cette saison."
    
    assert year in [2016, 2017, 2018, 2019, 2020]
    
    teams = [
        'Capitals', 'Penguins', 'Blue Jackets', 'Canadiens', 'Senators',
        'Bruins', 'Rangers', 'Maple Leafs', 'Islanders', 'Lightning',
        'Flyers', 'Hurricanes', 'Panthers', 'Red Wings', 'Sabres',
        'Devils', 'Blackhawks', 'Wild', 'Blues', 'Ducks', 'Oilers',
        'Sharks', 'Flames', 'Predators', 'Jets', 'Kings', 'Stars',
        'Coyotes', 'Canucks', 'Avalanche', 'Golden Knights'
    ]
    num_teams = len(teams)

    teams_dict = get_teams_dict(teams)

    # Define file path for the image
    filepath = f'../figures/{year}_{team}.png'
    
    # Check if the image already exists
    if os.path.exists(filepath):
        return filepath

    df = load_events_dataframe(year)
    df = df.query('zone_code == "O" or (zone_code == "N" and x_coord*goal_x_coord >= 0)')

    rates = np.zeros((num_teams + 1, 468, 551))
    last_gid = 0
    number_of_games = [0] * (num_teams + 1)

    for i in range(len(df)):
        plus_one_game = False
        point = df.iloc[i]
        gid = point["game_id"]
        x = float(point["x_coord"])
        y = float(point["y_coord"])

        if pd.isna(x) or pd.isna(y):
            continue

        if gid != last_gid: 
            number_of_games[-1] += 1
            last_gid = gid
            plus_one_game = True

        home_id = point["home_team_id"]
        away_id = point["away_team_id"]
        team_id = point["event_owner_team_id"]

        if plus_one_game:
            home_index = teams_dict[point["home_team_name"]]
            number_of_games[home_index] += 1
            away_index = teams_dict[point["away_team_name"]]
            number_of_games[away_index] += 1

        team_name = point["home_team_name"] if team_id == home_id else point["away_team_name"]
        team_index = teams_dict[team_name]

        if x < 0:
            x = -x
            y = -y 

        x_trans, y_trans = transform_coordinates(x, y)
        x = round(x_trans)
        y = round(y_trans)
        rates[-1][y][x] += 1
        rates[team_index][y][x] += 1

    number_of_games[-1] = number_of_games[-1] * 2
    number_of_games = put_right_format(number_of_games)
    rates = np.divide(rates, number_of_games)

    final_rates = np.subtract(rates[0:num_teams], rates[-1])

    smoothed_rates = []
    extremum = 0
    for i in range(len(final_rates)):
        rates = final_rates[i]
        smoothed_rates.append(gaussian_filter(rates, sigma=12))
        potential_extremum = np.max(np.abs(smoothed_rates))
        if potential_extremum > extremum:
            extremum = potential_extremum

    team_index = teams_dict[team]
    smoothed_rate = smoothed_rates[team_index]
    x = np.linspace(0, 551, 551)
    y = np.linspace(0, 468, 468)
    X, Y = np.meshgrid(x, y)

    levels = np.linspace(-extremum, extremum, 8)
    cmap = plt.get_cmap('coolwarm')
    norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)

    img = np.asarray(Image.open(rink_image_path))
    plt.imshow(img)
    visu = plt.contourf(X, Y, smoothed_rate, levels=levels, cmap=cmap, norm=norm, alpha=0.8)
    cbar = plt.colorbar(visu, ticks=levels)
    cbar.ax.set_ylabel('Shot Rate Difference per Hour')
    tick_locs = (levels[:-1] + levels[1:]) / 2
    cbar.set_ticks(tick_locs)
    plt.title(f'Shot rate difference for the {team}\nwith the mean for the league in {year}')
    plt.savefig(filepath)
    plt.clf()

    return filepath

def display_image(filepath):
    if filepath:
        with open(filepath, "rb") as file:
            img = file.read()
        img_widget = widgets.Image(value=img, format='png', width=500, height=400)
        display(img_widget)

# Dropdowns for season and team selection
def update_image(change):
    year = season_dropdown.value
    team = team_dropdown.value
    filepath_or_message = get_adv_visuals(year, team)
    
    clear_output(wait=False)  # Clear previous outputs but keep widgets
    display(season_dropdown, team_dropdown, message_output)  # Redisplay the dropdowns and message area
    
    if isinstance(filepath_or_message, str):
        if filepath_or_message.endswith('.png'):
            display_image(filepath_or_message)  # Display the new image as a widget
            message_output.value = ""  # Clear message area if image is displayed
        else:
            message_output.value = filepath_or_message  # Display the message if no image
    else:
        message_output.value = ""  # Clear the message if no valid image or message

season_dropdown = widgets.Dropdown(
    options=[2016, 2017, 2018, 2019, 2020],
    description='Season:',
    value=2019
)

team_dropdown = widgets.Dropdown(
    options=[
        'Capitals', 'Penguins', 'Blue Jackets', 'Canadiens', 'Senators',
        'Bruins', 'Rangers', 'Maple Leafs', 'Islanders', 'Lightning',
        'Flyers', 'Hurricanes', 'Panthers', 'Red Wings', 'Sabres',
        'Devils', 'Blackhawks', 'Wild', 'Blues', 'Ducks', 'Oilers',
        'Sharks', 'Flames', 'Predators', 'Jets', 'Kings', 'Stars',
        'Coyotes', 'Canucks', 'Avalanche', 'Golden Knights'
    ],
    description='Team:',
    value='Capitals'
)

# Output widget to display messages
message_output = widgets.HTML(value="")

season_dropdown.observe(update_image, names='value')
team_dropdown.observe(update_image, names='value')

# Display the dropdowns, message, and the initial image
display(season_dropdown, team_dropdown, message_output)
initial_filepath = get_adv_visuals(season_dropdown.value, team_dropdown.value)
if isinstance(initial_filepath, str) and initial_filepath.endswith('.png'):
    display_image(initial_filepath)
else:
    message_output.value = initial_filepath


Dropdown(description='Season:', options=(2016, 2017, 2018, 2019, 2020), value=2016)

Dropdown(description='Team:', index=30, options=('Capitals', 'Penguins', 'Blue Jackets', 'Canadiens', 'Senator…

HTML(value='')